Import All Required Packages for DEMO

In [1]:
import os
import numpy as np
import plotly.graph_objs as go
import plotly
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid, save_image
from torch.nn import functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from scipy.stats import norm
import scipy.io
import os
from math import ceil
import joblib
from sklearn import preprocessing
import plotly.express as px
import plotly.graph_objects as go

device = ( 'cuda' if torch.cuda.is_available() else 'cpu' )

In [2]:
!pip install dash # The core dash backend
!pip install dash-daq

# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.6
    Uninstalling Werkzeug-2.3.6:
      Successfully uninstalled Werkzeug-2.3.6
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.7/642.7 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dash-daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669696 sha256=e097862ffa3731e3f168d8295af46f0744e7e2e1abf343d9bfd838f0d0583572
  Stored in directory: /root/.cache/pip/wheels/75/14/1b/208d09d5e239391048bdc167759977b41ba65a3d4063aebf6b
Successfully built dash-daq


Autoencoder Definition

In [3]:
# Autoencoder

# Hyperparameters
batch_size = 128
device = device = ( 'cuda' if torch.cuda.is_available() else 'cpu' )

img_size1 = 138    # 28x28x1
hidden_dim = 64  # hidden layer dim
#z_dim = 20        # latent space dimension (encoder)
z_dim = 2        # latent space dimension (encoder)

epochs = 10

# model architecture
class VAE(nn.Module):
  def __init__(self):
    super(VAE, self).__init__()

    # encoder
    self.fc1 = nn.Linear(img_size1, hidden_dim)
    self.fc2_mean = nn.Linear(hidden_dim, z_dim)
    self.fc2_logvar = nn.Linear(hidden_dim, z_dim)
    #decoder
    self.fc3 = nn.Linear(z_dim, hidden_dim)
    self.fc4 = nn.Linear(hidden_dim, img_size1)

  def encode(self, x):
    h = F.relu(self.fc1(x))
    mu = self.fc2_mean(h)         # compute mean of latent
    logvar = self.fc2_logvar(h)   # compute logvar of latent
    return mu, logvar

  def reparameterize(self, mu, logvar):
    # p(z|x) = mu + std * eps
    std = torch.exp(logvar/2)
    eps = torch.randn_like(std)
    return (mu + eps * std)

  def decode(self, z):
    h = F.relu(self.fc3(z))
    out = torch.sigmoid(self.fc4(h))
    return out

  def forward(self, x):
    # batch_size x 1 x 28 x 28 -> batch_size x 784
    #print(x.view(-1, img_size).shape)
    mu, logvar = self.encode(x.view(-1, img_size1))
    z = self.reparameterize(mu, logvar)
    reconstructed = self.decode(z)
    return reconstructed, mu, logvar

# Initialize model, optimizer

# combined VAE
#img_size = 650
#totVae = VAE().to(device)
#totVae.load_state_dict(torch.load('/content/drive/MyDrive/video_feat/features/autoenc.pth'))

# Audio only VAE
#img_size = 138
audVae = VAE().to(device)
audVae.load_state_dict(torch.load('/content/drive/MyDrive/video_feat/audio_only/autoenc_audio_only.pth'))

<All keys matched successfully>

In [4]:
# Autoencoder

# Hyperparameters
batch_size = 128
device = device = ( 'cuda' if torch.cuda.is_available() else 'cpu' )

img_size = 650    # 28x28x1
hidden_dim = 64  # hidden layer dim
#z_dim = 20        # latent space dimension (encoder)
z_dim = 2        # latent space dimension (encoder)

epochs = 10

# model architecture
class VAE(nn.Module):
  def __init__(self):
    super(VAE, self).__init__()

    # encoder
    self.fc1 = nn.Linear(img_size, hidden_dim)
    self.fc2_mean = nn.Linear(hidden_dim, z_dim)
    self.fc2_logvar = nn.Linear(hidden_dim, z_dim)
    #decoder
    self.fc3 = nn.Linear(z_dim, hidden_dim)
    self.fc4 = nn.Linear(hidden_dim, img_size)

  def encode(self, x):
    h = F.relu(self.fc1(x))
    mu = self.fc2_mean(h)         # compute mean of latent
    logvar = self.fc2_logvar(h)   # compute logvar of latent
    return mu, logvar

  def reparameterize(self, mu, logvar):
    # p(z|x) = mu + std * eps
    std = torch.exp(logvar/2)
    eps = torch.randn_like(std)
    return (mu + eps * std)

  def decode(self, z):
    h = F.relu(self.fc3(z))
    out = torch.sigmoid(self.fc4(h))
    return out

  def forward(self, x):
    # batch_size x 1 x 28 x 28 -> batch_size x 784
    #print(x.view(-1, img_size).shape)
    mu, logvar = self.encode(x.view(-1, img_size))
    z = self.reparameterize(mu, logvar)
    reconstructed = self.decode(z)
    return reconstructed, mu, logvar

totVae = VAE().to(device)
totVae.load_state_dict(torch.load('/content/drive/MyDrive/video_feat/features/autoenc.pth'))

<All keys matched successfully>

In [5]:
print(totVae)

VAE(
  (fc1): Linear(in_features=650, out_features=64, bias=True)
  (fc2_mean): Linear(in_features=64, out_features=2, bias=True)
  (fc2_logvar): Linear(in_features=64, out_features=2, bias=True)
  (fc3): Linear(in_features=2, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=650, bias=True)
)


In [6]:
# VideoAudio ----------------
encoderVA = preprocessing.LabelEncoder()
encoderVA.classes_ = np.load('/content/drive/MyDrive/video_feat/features/classes.npy')

# these are the encoded video clips
songs_z_va = pd.read_csv('/content/drive/MyDrive/video_feat/features/audioVideo_z.csv')
songs_z_va.head()

# Audio ----------------
encoderAU = preprocessing.LabelEncoder()
encoderAU.classes_ = np.load('/content/drive/MyDrive/video_feat/audio_only/classes.npy')

# these are the encoded video clips
songs_z_au = pd.read_csv('/content/drive/MyDrive/video_feat/audio_only/audio_z.csv')
songs_z_au.head()

,x,y,labels
0,25.158928,1.049391,9
1,-2.364750,0.553579,7
2,-5.552360,0.839206,0
3,0.643493,0.635634,2
4,-1.848031,0.663291,4


Nearest Neighbors

In [7]:
from sklearn.neighbors import NearestNeighbors

# VA
neigh_va = NearestNeighbors(n_neighbors=5)
neigh_va.fit(songs_z_va[['x', 'y']].values)

# Au
neigh_au = NearestNeighbors(n_neighbors=5)
neigh_au.fit(songs_z_au[['x', 'y']].values)

NearestNeighbors()

'Toy' - Demo files

In [8]:
testdat1 = torch.rand(650)
testdat2 = torch.randn(650)
testdat3 = testdat2[:138]

In [9]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference


def similarityExtract(file, mode):

  if mode == 'AV':
    #print(totVae)
    _, mu, logvar = totVae(file)

    epsilon = 0.1
    std = torch.exp(logvar/2)
    eps = torch.randn_like(std)
    z = (mu + epsilon * std).data.cpu().numpy()
    #z = audioVae.reparameterize(mu, logvar).data.cpu().numpy()
    neighbor_distance, neighbor_index = neigh_va.kneighbors(z, return_distance=True)
    neighbor_color = neighbor_index
    #print(neighbor_color)
    plot_radius = float(np.array(np.max(neighbor_distance)))
    #print(f'pr {plot_radius}')
    ha = (neighbor_distance.reshape(-1, 1)+100)

    neighbor_distance = softmax(ha).round(3)*100
    neighbor_distance = softmax(100 - neighbor_distance).round(3)*100
    #print(neighbor_distance)
    neighbor_index = encoderVA.inverse_transform(neighbor_index.reshape(-1))
    #print(neighbor_index)
    print()
    print()
    for i in range(len(neighbor_index)):
      print(f'track{i+1}: {neighbor_index[i]}, probability: {neighbor_distance[i]}')

    fig1 = px.scatter(x=songs_z_va.iloc[:, 0], y=songs_z_va.iloc[:, 1])
    fig2 = px.scatter(x=z[:, 0], y=z[:, 1], color_discrete_sequence=['red'])
    #print(list(neighbor_color.reshape(-1)))
    z4 = songs_z_va.iloc[list(neighbor_color.reshape(-1))]
    #print(z4)
    fig4 = px.scatter(x=z4['x'], y=z4['y'], color =neighbor_index) #color_discrete_sequence=['LightGreen'])
    fig4.update_traces(
      marker=dict(size=8, symbol="star-diamond", line=dict(width=2, color="DarkSlateGrey")),
      selector=dict(mode="markers"),
    )


    fig3 = go.Figure(data=fig1.data + fig2.data + fig4.data)
    print(f'lp {z[:, 0]- plot_radius}')
    fig3.add_shape(type="circle", xref="x", yref="y", x0=float(z[:, 0])-plot_radius,
                  y0=float(z[:, 1])-plot_radius, x1=float(z[:, 0])+plot_radius, y1=float(z[:, 1])+plot_radius,
                  line_color="LightGreen", fillcolor="LightSalmon", opacity=0.2,)

    # Set figure size
    #fig3.update_layout(width=400, height=400)
    fig3.update_yaxes(
      scaleanchor="x",
      scaleratio=1,
    )

    #fig3.show()
    #return fig3

  if mode == 'A':

    file = file[:138]
    _, mu, logvar = audVae(file)

    epsilon = 0.1
    std = torch.exp(logvar/2)
    eps = torch.randn_like(std)
    z = (mu + epsilon * std).data.cpu().numpy()
    #z = audioVae.reparameterize(mu, logvar).data.cpu().numpy()
    neighbor_distance, neighbor_index = neigh_au.kneighbors(z, return_distance=True)
    neighbor_color = neighbor_index
    #print(neighbor_color)
    plot_radius = float(np.array(np.max(neighbor_distance)))
    #print(f'pr {plot_radius}')
    ha = (neighbor_distance.reshape(-1, 1)+100)

    neighbor_distance = softmax(ha).round(3)*100
    neighbor_distance = softmax(100 - neighbor_distance).round(3)*100
    #print(neighbor_distance)
    neighbor_index = encoderAU.inverse_transform(neighbor_index.reshape(-1))
    #print(neighbor_index)
    print()
    print()
    for i in range(len(neighbor_index)):
      print(f'track{i+1}: {neighbor_index[i]}, probability: {neighbor_distance[i]}')

    fig1 = px.scatter(x=songs_z_au.iloc[:, 0], y=songs_z_au.iloc[:, 1])
    fig2 = px.scatter(x=z[:, 0], y=z[:, 1], color_discrete_sequence=['red'])
    #print(list(neighbor_color.reshape(-1)))
    z4 = songs_z_au.iloc[list(neighbor_color.reshape(-1))]
    #print(z4)
    fig4 = px.scatter(x=z4['x'], y=z4['y'], color =neighbor_index) #color_discrete_sequence=['LightGreen'])
    fig4.update_traces(
      marker=dict(size=8, symbol="star-diamond", line=dict(width=2, color="DarkSlateGrey")),
      selector=dict(mode="markers"),
    )


    fig3 = go.Figure(data=fig1.data + fig2.data + fig4.data)
    print(f'lp {z[:, 0]- plot_radius}')
    fig3.add_shape(type="circle", xref="x", yref="y", x0=float(z[:, 0])-plot_radius,
                  y0=float(z[:, 1])-plot_radius, x1=float(z[:, 0])+plot_radius, y1=float(z[:, 1])+plot_radius,
                  line_color="LightGreen", fillcolor="LightSalmon", opacity=0.2,)

    # Set figure size
    #fig3.update_layout(width=400, height=400)
    fig3.update_yaxes(
      scaleanchor="x",
      scaleratio=1,
    )

    #fig3.show()
  return fig3

In [10]:
#similarityExtract(testdat3, 'A')

Demo Code

In [11]:
!pip install dash # The core dash backend
!pip install dash-daq

In [12]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

In [13]:


#df = px.data.gapminder().query("year==2007")

app = Dash(__name__)

app.layout = html.Div(
    [
        html.H4("Similarity Visualize"),
        html.P("Select file:"),
        dcc.RadioItems(
            id="pos-x",
            inline=True,
            options=["file1", "file2", "file3"],
            value="file1",
        ),
        dcc.RadioItems(
            id="pos-y",
            inline=True,
            options=["AV", "A"],
            value="AV",
        ),
        dcc.Graph(id="graph"),
    ]
)


@app.callback(
    Output("graph", "figure"),
    Input("pos-x", "value"),
    Input("pos-y", "value"),
)

def get_results(pos_x, pos_y):

  fpath = testdat1
  if pos_x == 'file1':
    fpath = testdat1
  if pos_x == 'file2':
    fpath = testdat2
  if pos_x == 'file3':
    fpath = testdat3


  fig = similarityExtract(fpath, pos_y)
  return fig


if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>